Based off the resource at https://neo4j.com/docs/graph-data-science-client/current/tutorials/load-data-via-graph-construction/

In [4]:
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience # Load neo4j graph data science library
import os
if os.getcwd().split('\\')[-1] == 'notebooks':
    os.chdir('..')

In [5]:
# Connect to a Neo4j instance (assumes local right now)
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = None
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )
else:
    NEO4J_AUTH = ("neo4j", "Bookings")
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

In [6]:
# Load the data
properties = pd.read_csv("Inputs/property_data.csv")
bookings = pd.read_csv("Inputs/bookings_data.csv")

In [7]:
bookings.head()

,year,week_num,property_type,country,city,capacity,pets_allowed,property_id,booked_through,booking_id
0,2019,34,apartment,ITALY,Milan,4,yes,342,3rd Party,B-0
1,2019,46,apartment,BELGIUM,Brussels,4,no,909,3rd Party,B-1
2,2019,13,apartment,BELGIUM,Brussels,6,yes,7,3rd Party,B-2
3,2020,25,condo,SPAIN,Madrid,7,no,263,3rd Party,B-3
4,2020,25,house,SPAIN,Barcelona,6,yes,262,3rd Party,B-4


In [8]:
# Start with a fresh database
gds.run_cypher("MATCH (n) DETACH DELETE n")

# Create Database Constraints for uniqueness to make sure we load properly
gds.run_cypher("""
CREATE CONSTRAINT unique_properties IF NOT EXISTS
FOR (p:Property) REQUIRE p.id IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_countries IF NOT EXISTS
FOR (c:Country) REQUIRE c.name IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_cities IF NOT EXISTS
FOR (c:City) REQUIRE c.name IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_property_types IF NOT EXISTS
FOR (t:Type) REQUIRE t.type IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_bookings IF NOT EXISTS
FOR (b:Booking) REQUIRE b.booking_id IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_weeks IF NOT EXISTS
FOR (w:Week) REQUIRE w.week_num IS UNIQUE
""")

""


## Load property information
Loop through the rows of the properties data (as a list of dictionaries) and
- Create a node for each property in the graph database.
- Set the capacity and pets_allowed attributes
- Create the type, city, and country nodes if they do not already exist
- Create relationships between the property and the type as well as the property and the city
- Create relationships between the city and the country

In [9]:
property_setup_query = """
UNWIND $properties AS property
MERGE (p:Property {id: property.id})
SET p.capacity = property.capacity
SET p.pets_allowed = property.pets_allowed
MERGE (t:Type {name: property.property_type})
MERGE (ct:City {name: property.city})
MERGE (c:Country {name: property.country})
MERGE (p)-[:LOCATED_IN]->(ct)
MERGE (ct)-[:LOCATED_IN]->(c)
MERGE (p)-[:HAS_TYPE]->(t)
"""

## Load the Bookings
Loop through the bookings and
- Find the property for the booking
- Create the node for the booking
- Set the year and booked_through properties
- Create a node for the week if it doesn't already exist
- Create a relationship between the booking and the property
- Create a relationship between the booking and the week

In [10]:
bookings_setup_query = """
UNWIND $bookings AS booking
MATCH (p:Property {id: booking.property_id})
MERGE (b:Booking {booking_id: booking.booking_id})
SET b.year = booking.year
SET b.booked_through = booking.booked_through
MERGE (w:Week {week_num: booking.week_num})
MERGE (b)-[:BOOKED_IN]->(w)
MERGE (b)-[:BOOKED_AT]->(p)
"""

In [11]:
# Convert dataframes to dictionaries
property_data_dict = {"properties": properties.to_dict('records')}
bookings_data_dict = {"bookings": bookings[['booking_id', 'booked_through', 'year', 'week_num', 'property_id']].to_dict('records')}

In [12]:
# Run the queries to load the data
gds.run_cypher(property_setup_query, property_data_dict)
gds.run_cypher(bookings_setup_query, bookings_data_dict)

""


In [13]:
# Build relationships across weeks (cureent week to next week)
gds.run_cypher("""
MATCH (w:Week)
MATCH (w_p: Week)
WHERE w.week_num = w_p.week_num - 1
MERGE (w)-[:PRECEDES]->(w_p)
""")

# Account for edge case of end of year to complete the loop
gds.run_cypher("""
MATCH (w:Week)
MATCH (w_p: Week)
WHERE w.week_num = 52 AND w_p.week_num = 1
MERGE (w)-[:PRECEDES]->(w_p)
""")

""


In [14]:
# Create Train/Test Split (Separate using year such that all of 2024 is the holdout period)
gds.run_cypher("""
MATCH (p:Property)<-[:BOOKED_AT]-(b:Booking)-[:BOOKED_IN]->(w:Week)
WHERE b.year < 2020
MERGE (p)-[:TRAIN_BOOKING]->(w)
""")

gds.run_cypher("""
MATCH (p:Property)<-[:BOOKED_AT]-(b:Booking)-[:BOOKED_IN]->(w:Week)
WHERE b.year >= 2020
MERGE (p)-[:HOLDOUT_BOOKING]->(w)
""")

""
